#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2")

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
customer_csv = """101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
"""

usage_csv = """customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
"""

tower_logs_region1 = """event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
"""
tower_logs_region2 = """event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
"""
cust_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/cust.csv"
dbutils.fs.put(cust_path, customer_csv, True)

us_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv"
dbutils.fs.put(us_path, usage_csv, True)

tower_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv"
dbutils.fs.put(tower_path, tower_logs_region1, True)

tower_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region2.csv"
dbutils.fs.put(tower_path, tower_logs_region2, True)

str="id int,name string,age int,city string,plan string"
cust_df = spark.read.schema(str).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/cust.csv", inferSchema=True, header=True)
print(cust_df.printSchema())
cust_df.show(10)


In [0]:
usage_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", inferSchema=True, header=True, sep='\t')
print(usage_df.printSchema())
usage_df.show(10)

In [0]:
tower_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv", inferSchema=True, header=True, sep='|')
print(tower_df.printSchema())
tower_df.show(10)

In [0]:
df_multiple_sources=spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/"],inferSchema=True,header=True,sep='|',pathGlobFilter="tower_logs_region*",recursiveFileLookup=True)
df_multiple_sources.show(10)

In [0]:
df2=spark.read.option("header", "True").option("recursiveFileLookup","true").option("pathGlobFilter", "*.csv").csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/","Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*/"])
display(df2)
print(df2.printSchema())

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
usage_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", inferSchema=False, header=False, sep='\t')
print(usage_df.printSchema())
usage_df.show(10)

str="id int,name string,age int,city string,plan string, corrupt_record string"
cust_df = spark.read.schema(str).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/cust.csv", inferSchema=True, header=True, mode="PERMISSIVE", columnNameOfCorruptRecord="corrupt_record")
print(cust_df.printSchema())
cust_df.show(10)

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
#str="id int,name string,age int,city string,plan string, corrupt_record string"
cust_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/cust.csv", inferSchema=True, header=True, mode="PERMISSIVE").toDF("id","name","age","city","plan")
print(cust_df.printSchema())
cust_df.show(10)

str="cust_id int,voice_mn int,mb_data int,sms_cnt int"
usage_df = spark.read.schema(str).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", inferSchema=True, header=True, sep='\t')
print(usage_df.printSchema())
usage_df.show(10)

from pyspark.sql.types import *
custom_schema=StructType([StructField("event_id",IntegerType(),False),StructField("cust_id",IntegerType(),False),StructField("tower_id",StringType(),False),StructField("signal_strength",IntegerType(),False),StructField("timestamp",TimestampType(),False)])
tower_df = spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv", inferSchema=True, header=True, sep='|')
print(tower_df.printSchema())
tower_df.show(10)

## 6. More to come (stay motivated)....